In [1]:
# Cargar librerías necesarias
import json
from azureml.core import Workspace, Model, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [5]:
# Open config from .json file
with open('keys.json') as config_file:
    config = json.load(config_file)

# Cargar id
id = config['Azure_ID']

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="Instituto Tecnologico y de Estudios Superiores de Monterrey (tecmx.onmicrosoft.com)") 

# Crear el workspace de Azure
ws = Workspace.create(name="Sales-LT-ML",
                   subscription_id=id,
                   resource_group='class-servicio-sql',
                   location ='centralindia',
                   auth = interactive_auth) # centralindia

UserErrorException: UserErrorException:
	Message: You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 85122af0-96c1-4e25-b6c8-b59805cf664f subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Visual Studio Enterprise Subscription Cad', subscription_id='90eaa2d9-15dc-4bc3-9c4f-2c580923e179'), SubscriptionInfo(subscription_name='Azure subscription 1', subscription_id='cf842899-b4df-4f27-ba04-2e357f216505')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 85122af0-96c1-4e25-b6c8-b59805cf664f subscription, please check if it is in this tenant. All the subscriptions that you have access t

In [ ]:
# Cargar el workspace de Azure
ws = Workspace.get(name='Sales-LT-ML', subscription_id=id, resource_group='class-servicio-sql', location='centralindia')

In [ ]:
ModelName = 'MyModel'
RegisteredModel = Model.register(model_path='model.pkl',model_name=ModelName, workspace=ws)

In [ ]:
scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('MyModel')
    model = joblib.load(model_path)

def run(data):
    try:
        data = json.loads(data)
        df = pd.DataFrame(data['data'])

        

        # Predicción
        result = model.predict(df).tolist()
        return json.dumps(result)
    except Exception as e:
        return json.dumps({{"error": str(e)}})
"""


# Guardar el archivo en la subcarpeta /scripts
with open('scripts/score.py', 'w') as f:
    f.write(scorepy)
